 1. Imports 

In [4]:
# Imports
import pandas as pd
import mysql.connector
import os

Data base connection

In [ ]:
# --- DB connection ---
connection = mysql.connector.connect(
    host='euro-election-trends-db-mysql-aminesadik969-b6d6.g.aivencloud.com',
    user='avnadmin',
    password='check  whats up group',
    database='defaultdb',
    port ='24637'
)

Load data from files

In [6]:

cursor = connection.cursor()

# --- Load Excel ---
excel_path = './data/2019.xlsx'
xls = pd.ExcelFile(excel_path)
df_provincias = xls.parse('Provincias', header=None)


Insert data feched from files to sal db in aiven

In [7]:


# --- Insert into Partido ---
nombres = df_provincias.loc[3, 16:]
siglas = df_provincias.loc[4, 16:]

for sigla, nombre in zip(siglas, nombres):
    if pd.isna(sigla) or pd.isna(nombre):
        continue

    cursor.execute("SELECT ID_Partido FROM Partido WHERE Siglas = %s", (sigla,))
    if cursor.fetchone() is None:
        cursor.execute("INSERT INTO Partido (Nombre, Siglas) VALUES (%s, %s)", (nombre, sigla))
        print(f"Inserted Partido: {sigla} - {nombre}")
    else:
        print(f"Partido exists: {sigla}")

connection.commit()

# --- Insert into Elecciones ---
filename = os.path.basename(excel_path)
anio = int(filename.split('.')[0])
tipo = 'Europeas'

cursor.execute("SELECT ID_Elecciones FROM Elecciones WHERE Anio = %s AND Tipo = %s", (anio, tipo))
result = cursor.fetchone()
if result:
    id_elecciones = result[0]
else:
    cursor.execute("INSERT INTO Elecciones (Anio, Tipo) VALUES (%s, %s)", (anio, tipo))
    connection.commit()
    id_elecciones = cursor.lastrowid
print(f"Using ID_Elecciones = {id_elecciones}")

# --- Insert into Comunidad and Provincia ---
for idx in range(6, df_provincias.shape[0]):
    comunidad = df_provincias.iloc[idx, 0]
    provincia = df_provincias.iloc[idx, 1]

    if pd.isna(comunidad) or pd.isna(provincia):
        continue

    # Comunidad
    cursor.execute("SELECT ID_Comunidad FROM Comunidad WHERE Nombre = %s", (comunidad,))
    result = cursor.fetchone()
    if result:
        comunidad_id = result[0]
    else:
        cursor.execute("INSERT INTO Comunidad (Nombre) VALUES (%s)", (comunidad,))
        connection.commit()
        comunidad_id = cursor.lastrowid

    # Provincia
    cursor.execute("SELECT ID_Provincia FROM Provincia WHERE Nombre = %s", (provincia,))
    result = cursor.fetchone()
    if result is None:
        cursor.execute(
            "INSERT INTO Provincia (Nombre, ID_Comunidad) VALUES (%s, %s)",
            (provincia, comunidad_id)
        )
        connection.commit()
        print(f"Inserted Provincia: {provincia} (Comunidad: {comunidad})")

# --- Insert into Resultados ---
for idx in range(5, df_provincias.shape[0]):
    provincia = df_provincias.iloc[idx, 1]
    if pd.isna(provincia):
        continue

    # Get provincia ID
    cursor.execute("SELECT ID_Provincia FROM Provincia WHERE Nombre = %s", (provincia,))
    result = cursor.fetchone()
    if not result:
        print(f"Provincia not found: {provincia}")
        continue
    provincia_id = result[0]

    # Loop through partidos
    for col_idx, sigla in siglas.items():
        if pd.isna(sigla):
            continue  # skip if sigla is NaN

        sigla = str(sigla).strip()
        if not sigla:
            continue  # skip empty strings

        votos = df_provincias.iloc[idx, col_idx]
        if pd.isna(votos):
            votos = 0  # default to 0 if empty

        # Get partido ID
        cursor.execute("SELECT ID_Partido FROM Partido WHERE Siglas = %s", (sigla,))
        result = cursor.fetchone()
        if not result:
            print(f"Partido not found: '{sigla}'")
            continue
        partido_id = result[0]

        # Check existence
        cursor.execute("""
            SELECT * FROM Resultados
            WHERE ID_Elecciones = %s AND ID_Provincia = %s AND ID_Partido = %s
        """, (id_elecciones, provincia_id, partido_id))

        if cursor.fetchone() is None:
            try:
                votos_int = int(votos)
            except (ValueError, TypeError):
                continue  # Skip rows where 'votos' is not a valid number

            cursor.execute("""
                INSERT INTO Resultados (ID_Elecciones, ID_Provincia, ID_Partido, Votos)
                VALUES (%s, %s, %s, %s)
            """, (id_elecciones, provincia_id, partido_id, votos_int))
            print(f"Inserted Result: {sigla} in {provincia} → {votos_int}")


# --- Insert into Diputados ---
last_row_idx = df_provincias.shape[0] - 1  # last row

for col_idx, sigla in siglas.items():
    if pd.isna(sigla):
        continue

    sigla = str(sigla).strip()
    if not sigla:
        continue

    try:
        diputados = int(df_provincias.iloc[last_row_idx, col_idx + 1])
    except (ValueError, TypeError):
        continue  # skip if not valid number

    # Get ID_Partido
    cursor.execute("SELECT ID_Partido FROM Partido WHERE Siglas = %s", (sigla,))
    result = cursor.fetchone()
    if not result:
        continue
    partido_id = result[0]

    # Check if already inserted
    cursor.execute("""
        SELECT * FROM Diputados
        WHERE ID_Elecciones = %s AND ID_Partido = %s
    """, (id_elecciones, partido_id))
    if cursor.fetchone() is None:
        cursor.execute("""
            INSERT INTO Diputados (ID_Elecciones, ID_Partido, Diputados)
            VALUES (%s, %s, %s)
        """, (id_elecciones, partido_id, diputados))
        print(f"Inserted total Diputados: {sigla} → {diputados}")

# --- Insert into Votantes ---

# Final commit
# --- Insert into Votantes ---
for idx in range(6, df_provincias.shape[0]):
    provincia = df_provincias.iloc[idx, 1]  # Column 1: Provincia
    if pd.isna(provincia):
        continue

    # Get provincia ID
    cursor.execute("SELECT ID_Provincia FROM Provincia WHERE Nombre = %s", (provincia,))
    result = cursor.fetchone()
    if not result:
        continue
    provincia_id = result[0]

    # Extract values with default 0 on failure
    def get_int(col):
        try:
            return int(df_provincias.iloc[idx, col])
        except (ValueError, TypeError):
            return 0

    poblacion = get_int(3)
    numero_mesas = get_int(4)
    censo_sin_cera = get_int(5)
    censo_cera = get_int(6)
    total_censo_electoral = get_int(7)
    solicitudes_cera_aceptadas = get_int(8)
    total_votantes_cer = get_int(9)
    total_votantes_cera = get_int(10)
    total_votantes = get_int(11)
    votos_validos = get_int(12)
    votos_candidaturas = get_int(13)
    votos_blanco = get_int(14)
    votos_nulos = get_int(15)

    # Check if record exists
    cursor.execute("""
        SELECT * FROM Votantes
        WHERE ID_Elecciones = %s AND ID_Provincia = %s
    """, (id_elecciones, provincia_id))

    if cursor.fetchone() is None:
        cursor.execute("""
            INSERT INTO Votantes (
                ID_Elecciones, ID_Provincia,
                Poblacion, Numero_Mesas, Censo_Sin_CERA, Censo_CERA,
                Total_Censo_Electoral, Solicitudes_CERA_Aceptadas,
                Total_Votantes_CER, Total_Votantes_CERA, Total_Votantes,
                Votos_Validos, Votos_Candidaturas, Votos_Blanco, Votos_Nulos
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_elecciones, provincia_id,
            poblacion, numero_mesas, censo_sin_cera, censo_cera,
            total_censo_electoral, solicitudes_cera_aceptadas,
            total_votantes_cer, total_votantes_cera, total_votantes,
            votos_validos, votos_candidaturas, votos_blanco, votos_nulos
        ))

        print(f"Inserted Votantes: {provincia}")


connection.commit()
cursor.close()
connection.close()


Inserted Partido: PSOE - PARTIDO SOCIALISTA OBRERO ESPAÑOL
Inserted Partido: PP - PARTIDO POPULAR
Inserted Partido: Cs - CIUDADANOS-PARTIDO DE LA CIUDADANÍA
Inserted Partido: PODEMOS-IU - UNIDAS PODEMOS CAMBIAR EUROPA
Inserted Partido: VOX - VOX
Inserted Partido: AHORA REPÚBLICAS - AHORA REPÚBLICAS
Inserted Partido: JUNTS - LLIURES PER EUROPA
Inserted Partido: CEUS - COALICIÓN POR UNA EUROPA SOLIDARIA
Inserted Partido: CPE - COMPROMÍS PER EUROPA /COMPROMISO POR EUROPA
Inserted Partido: PACMA - PARTIDO ANIMALISTA CONTRA EL MALTRATO ANIMAL
Inserted Partido: CV-EC - COALICIÓN VERDE-EUROPA CIUDADANA
Inserted Partido: RECORTES CERO-LV-GVE - RECORTES CERO-LOS VERDES-GRUPO VERDE EUROPEO
Inserted Partido: VOLT - VOLT EUROPA
Inserted Partido: I.Fem - INICIATIVA FEMINISTA
Inserted Partido: PCPE-PCPC-PCPA - PCPE-PCPC-PCPA
Inserted Partido: PACT - ACTÚA
Inserted Partido: AxSÍ - ANDALUCÍA POR SÍ
Inserted Partido: PUM+J - POR UN MUNDO MÁS JUSTO
Inserted Partido: PCTE - PARTIDO COMUNISTA DE LOS TRABA